<a href="https://colab.research.google.com/github/EvenSol/NeqSim-Colab/blob/master/notebooks/process/reservoirGORprocesstuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install neqsim

#Oil and Gas Reference Process Using NeqSim

#1. Create the fluid
In this case we establish a initial reservoir fluid. The fluid will typically be created from Eclipse fluid description files (E300). For simplicity we estrablish a simple fluid in this workbook.

In [2]:
import pandas as pd
from neqsim.thermo import fluid_df,TPflash,printFrame
# Read gas-oil process from file

# Uncomment to create alternative well stream
wellStreamComposition = {'ComponentName':  ["nitrogen", "CO2", "methane", "ethane", "propane", "i-butane", "n-butane", "i-pentane", "n-pentane", "n-hexane", "C7", "C8", "C9", "C10", "C11", "C12", "C13", "C14", "C15", "C16", "C17", "C18", "C19", "C20"],
        'MolarComposition[-]':  [0.53, 3.3, 72.98, 7.68, 4.1, 0.7, 1.42, 0.54, 0.67, 0.85, 1.33, 1.33, 0.78, 0.61, 0.42, 0.33, 0.42, 0.24, 0.3, 0.17, 0.21, 0.15, 0.15, 0.8],
        'MolarMass[kg/mol]': [None,None, None,None,None,None,None,None,None,None,0.0913, 0.1041, 0.1188, 0.136, 0.150, 0.164, 0.179, 0.188, 0.204, 0.216, 0.236, 0.253, 0.27, 0.391],
        'RelativeDensity[-]': [None,None, None,None,None,None,None,None,None,None, 0.746, 0.768, 0.79, 0.787, 0.793, 0.804, 0.817, 0.83, 0.835, 0.843, 0.837, 0.84, 0.85, 0.877]
  }

wellStreamCompositiondf = pd.DataFrame(wellStreamComposition)
wellStream = fluid_df(wellStreamCompositiondf)
wellStream.setMixingRule('classic')

wellStream.setTemperature(15.0, 'C')
wellStream.setPressure(1.0, 'atm')
TPflash(wellStream)

gor_one_Stage_process = wellStream.getPhase('gas').getFlowRate('Sm3/hr')/(wellStream.getPhase('oil').getFlowRate('m3/hr'))
print('GOR one stage flash ', gor_one_Stage_process)
printFrame(wellStream)


GOR one stage flash  1489.182661194047
| 0                    | 1          | 2           | 3           | 4   | 5   | 6               |
|:---------------------|:-----------|:------------|:------------|:----|:----|:----------------|
|                      | total      | GAS         | OIL         |     |     |                 |
| nitrogen             | 5.29947E-3 | 5.71188E-3  | 7.34389E-6  |     |     | [mole fraction] |
| CO2                  | 3.29967E-2 | 3.55226E-2  | 5.84197E-4  |     |     | [mole fraction] |
| methane              | 7.29727E-1 | 7.86281E-1  | 4.01184E-3  |     |     | [mole fraction] |
| ethane               | 7.67923E-2 | 8.25468E-2  | 2.94875E-3  |     |     | [mole fraction] |
| propane              | 4.09959E-2 | 4.37039E-2  | 6.24656E-3  |     |     | [mole fraction] |
| i-butane             | 6.9993E-3  | 7.33427E-3  | 2.70091E-3  |     |     | [mole fraction] |
| n-butane             | 1.41986E-2 | 1.46562E-2  | 8.32679E-3  |     |     | [mole fraction] |
|

# Reference process
In this exmaple we establish a three stage separation process for reference. Such a process can typically be used in as a basis for reservoir simulators when reporting gas and oil production. We need to simulate it to get the feed composition corresponding to a given GOR from the reservoir simulator.

In [19]:
from neqsim.process import recycle2, compressor, cooler, splitter, separator3phase, getProcess, clearProcess, compsplitter, mixer, heater, stream, pump, separator, runProcess, stream, saturator, valve, filters, heatExchanger, simpleTEGAbsorber,distillationColumn, waterStripperColumn, recycle2, setpoint, calculator

first_stage_pressure = 70.0
first_stage_temperature = 55.0

second_stage_pressure = 15.0
second_stage_temperature = 45.0

third_stage_pressure = 5.0
third_stage_temperature = 45.0

clearProcess()

feedToOffshoreProcess = stream(wellStream)
feedToOffshoreProcess.setName('feed to process')
feedToOffshoreProcess.setFlowRate(1000.0, 'kg/hr')

gasres= feedToOffshoreProcess.clone()
gasres.setFlowRate(0.001,'kg/hr')

oilres= feedToOffshoreProcess.clone()
oilres.setFlowRate(0.001,'kg/hr')

feedMixer = mixer("feed mixer")
feedMixer.addStream(feedToOffshoreProcess)
feedMixer.addStream(gasres)
feedMixer.addStream(oilres)

feedHeater = heater(feedMixer.getOutletStream())
feedHeater.setName("1st stage TP setter")
feedHeater.setOutPressure(first_stage_pressure, 'bara')
feedHeater.setOutTemperature(first_stage_temperature, 'C')

firstStageSeparator = separator(feedHeater.getOutStream())
firstStageSeparator.setName("1st stage separator")
firstStageSeparator.addStream(gasres)

oilHeaterFromFirstStage = heater(firstStageSeparator.getLiquidOutStream())
oilHeaterFromFirstStage.setName("2nd stage TP setter")
oilHeaterFromFirstStage.setOutPressure(second_stage_pressure, 'bara')
oilHeaterFromFirstStage.setOutTemperature(second_stage_temperature, 'C')

secondStageSeparator = separator(oilHeaterFromFirstStage.getOutStream())
secondStageSeparator.setName("2nd stage separator")

oilHeaterFrom_secondstage = heater(secondStageSeparator.getLiquidOutStream())
oilHeaterFrom_secondstage.setName("3rd stage TP setter")
oilHeaterFrom_secondstage.setOutPressure(third_stage_pressure, 'bara')
oilHeaterFrom_secondstage.setOutTemperature(third_stage_temperature, 'C')

thirdStageSeparator = separator(oilHeaterFrom_secondstage.getOutStream())
thirdStageSeparator.setName("3rd stage separator")

oilHeaterFrom_thirdstage = heater(thirdStageSeparator.getLiquidOutStream())
oilHeaterFrom_thirdstage.setName("standard condition TP setter")
oilHeaterFrom_thirdstage.setOutPressure(0.0, 'barg')
oilHeaterFrom_thirdstage.setOutTemperature(15.0, 'C')

standardSeparator = separator(oilHeaterFrom_thirdstage.getOutStream())
standardSeparator.setName("standard stage separator")

gasMixer = mixer("gas mixer")
gasMixer.addStream(firstStageSeparator.getGasOutStream())
gasMixer.addStream(secondStageSeparator.getGasOutStream())
gasMixer.addStream(thirdStageSeparator.getGasOutStream())
gasMixer.addStream(standardSeparator.getGasOutStream())


gas_split = splitter(gasMixer.getOutletStream())
gas_split.setName("gas splitter")
gas_split.setFlowRates([-1, 0.001], 'kg/hr')

oil_split = splitter(standardSeparator.getLiquidOutStream())
oil_split.setName("oil splitter")
oil_split.setFlowRates([-1, 0.001],'kg/hr')

total_stream_mixer = mixer('total mixer')
total_stream_mixer.addStream(standardSeparator.getLiquidOutStream())
total_stream_mixer.addStream(gasMixer.getOutStream())

total_stream = total_stream_mixer.getOutletStream()

oil_recylce_heater = heater(oil_split.getSplitStream(1))
oil_recylce_heater.setName("oil split heater")
oil_recylce_heater.setOutPressure(first_stage_pressure, 'bara')
oil_recylce_heater.setOutTemperature(first_stage_temperature, 'C')

gas_recylce_heater = heater(gas_split.getSplitStream(1))
gas_recylce_heater.setName("gas split heater")
gas_recylce_heater.setOutPressure(first_stage_pressure, 'bara')
gas_recylce_heater.setOutTemperature(first_stage_temperature, 'C')

gasresycle = recycle2()
gasresycle.addStream(gas_recylce_heater.getOutStream())
gasresycle.setName("gas recycle")
gasresycle.setOutletStream(gasres)
gasresycle.setTolerance(1e-10)

oilresycle = recycle2()
oilresycle.addStream(oil_recylce_heater.getOutStream())
oilresycle.setName("gas recycle")
oilresycle.setOutletStream(oilres)
oilresycle.setTolerance(1e-10)

runProcess()

def calc_process(rate):
  if(rate>=0):
      gas_split.setFlowRates([-1, rate], 'kg/hr')
      oil_split.setFlowRates([-1, 0.001],'kg/hr')
  else:
      gas_split.setFlowRates([-1, 0.001], 'kg/hr')
      oil_split.setFlowRates([-1, abs(rate)],'kg/hr')
  runProcess()
  gor_process = gasMixer.getOutletStream().getFluid().getPhase('gas').getFlowRate('Sm3/hr')/(standardSeparator.getLiquidOutStream().getFluid().getPhase('oil').getFlowRate('m3/hr'))
  return gor_process-target_gor

# Tuning the feed fluid composition to a given reference process GOR
We apply the fsolve function from scipy package to find the feed composition giving a specifiied GOR of the reference process.

In [20]:
import numpy as np
import scipy
from scipy.optimize import fsolve

target_gor = 1489.0
minimum=fsolve(calc_process, 30);
updated_gor_process = gasMixer.getOutletStream().getFluid().getPhase('gas').getFlowRate('Sm3/hr')/(standardSeparator.getLiquidOutStream().getFluid().getPhase('oil').getFlowRate('m3/hr'))
print('tuned gor ', updated_gor_process)

<ipython-input-19-323b3f061c70>:109: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  gas_split.setFlowRates([-1, rate], 'kg/hr')


tuned gor  1488.9999985605348


<ipython-input-20-1432fe855670>:6: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  minimum=fsolve(calc_process, 30);


# Show optimized stream
The final feed composition is stored in the total_stream variable.


In [22]:
print('gor reference process ', target_gor)
printFrame(total_stream.getFluid())
fluid_comp = list(total_stream.getFluid().getMolarComposition())
fluid_comp

wellStream = total_stream.getFluid().clone()
wellStream.setTemperature(15.0, 'C')
wellStream.setPressure(1.0, 'atm')
TPflash(wellStream)
gor_one_Stage_process = wellStream.getPhase('gas').getFlowRate('Sm3/hr')/(wellStream.getPhase('oil').getFlowRate('m3/hr'))
print('GOR one stage flash ', gor_one_Stage_process)

gor reference process  1489.0
| 0                    | 1          | 2                 | 3                 | 4   | 5   | 6               |
|:---------------------|:-----------|:------------------|:------------------|:----|:----|:----------------|
|                      | total      | GAS               | OIL               |     |     |                 |
| nitrogen             | 5.38441E-3 | 5.74442E-3        | 7.39047E-6        |     |     | [mole fraction] |
| CO2                  | 3.35141E-2 | 3.57159E-2        | 6.2953E-4         |     |     | [mole fraction] |
| methane              | 7.41411E-1 | 7.90773E-1        | 4.16581E-3        |     |     | [mole fraction] |
| ethane               | 7.78595E-2 | 8.28575E-2        | 3.21146E-3        |     |     | [mole fraction] |
| propane              | 4.08997E-2 | 4.31742E-2        | 6.92855E-3        |     |     | [mole fraction] |
| i-butane             | 6.80534E-3 | 7.06126E-3        | 2.983E-3          |     |     | [mole fraction] 

#Setting the total feed rate
After tuning GOR to a reference process the feed flow rate can be set as basis for the process calculation.

In [30]:
new_feed_stream = total_stream.copy()
new_feed_stream.setFlowRate(10.0, 'MSm3/day')
new_feed_stream.setPressure(100.0, 'bara')
new_feed_stream.setTemperature(70.0, 'C')
new_feed_stream.run()

json_report = new_feed_stream.getReport_json()
print(json_report)



{
  "liquidOutStream": {
    "name": "liquidOutStream",
    "properties": {
      "oil": {
        "relative density": {
          "value": "0.675293602532151",
          "unit": "-"
        },
        "heat capacity (Cv)": {
          "value": "2.00271222218418",
          "unit": "kJ/kgK"
        },
        "molar mass": {
          "value": "0.09450173542728059",
          "unit": "kg/mol"
        },
        "entropy": {
          "value": "-392.02549702499",
          "unit": "J/kgK"
        },
        "density": {
          "value": "675.293602532151",
          "unit": "kg/m3"
        },
        "enthalpy": {
          "value": "-215815.4218048122",
          "unit": "J/kg"
        },
        "flow rate": {
          "value": "325.446268006465",
          "unit": "m3/hr"
        },
        "RVP": {
          "value": "160.12038446641392",
          "unit": "bara"
        },
        "heat capacity (Cp)": {
          "value": "2.3434834401683693",
          "unit": "kJ/kgK"
       